In [1]:
import json

In [ ]:
''' We write a query to query the openstreetmaps api for all roads within a given bounding box.
The bounding box we have given is restricted only to all of melbourne'''

!curl --globoff 'http://overpass-api.de/api/interpreter?[out:json];\
(way["highway"~"motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|unclassified|unclassified_link|residential|residential_link|living_street"]\
(-38.5632,143.9236,-37.3396,146.7142););(._;node(w););out;' >> interpreter.json

with open("interpreter.json","r+") as content:
    data = json.load(content)

In [2]:
# get all nodes
node_list = [ el for el in data["elements"] if el["type"]=="node"]

# get all ways
way_list = [ el for el in data["elements"] if el["type"]=="way"]

# dict of coordinates for each node
coords = { el["id"]:(el["lon"],el["lat"]) for el in node_list}

del data , node_list

The code below allows us to get all nodes we can directly reach from a particular node. 

In [7]:
state_actions = dict()

for element in way_list:
    nodes = element["nodes"]
    nodes = [coords[x] for x in nodes]


    tags = element["tags"]
    n = len(nodes) 
    # if one way we only add node a >> node b
    if "oneway" in tags.keys() and tags["oneway"]=="yes":
        for i in range(n-1):
            if nodes[i] in state_actions.keys():
                state_actions[ nodes[i] ].append(nodes[i+1])
            else:
                state_actions[ nodes[i] ] = [nodes[i+1]]
    # if not one way we add node a >> node b and node b >> node a
    else:
        for i in range(n-1):
            if nodes[i] in state_actions.keys():
                state_actions[ nodes[i] ].append(nodes[i+1])
            else:
                state_actions[ nodes[i] ] = [nodes[i+1]]
            if nodes[i+1] in state_actions.keys():
                state_actions[ nodes[i+1] ].append(nodes[i])
            else:
                state_actions[ nodes[i+1] ] = [nodes[i]]

In [9]:
# save the output
s_a = { str(k):v for k,v in state_actions.items()}
with open('state_action.json','w') as out:
    json.dump(s_a,out)